In [24]:
import os
from sklearn.metrics import plot_roc_curve, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [25]:
os.environ['QT_QPA_PLATFORM'] = 'offscreen'

In [26]:
def import_data(pth):
    '''
    returns dataframe for the csv found at pth
    input:
            pth: a path to the csv
    output:
            df: pandas dataframe
    '''
    return pd.read_csv(pth)


cat_columns = [
    'Gender',
    'Education_Level',
    'Marital_Status',
    'Income_Category',
    'Card_Category'
]

quant_columns = [
    'Customer_Age',
    'Dependent_count',
    'Months_on_book',
    'Total_Relationship_Count',
    'Months_Inactive_12_mon',
    'Contacts_Count_12_mon',
    'Credit_Limit',
    'Total_Revolving_Bal',
    'Avg_Open_To_Buy',
    'Total_Amt_Chng_Q4_Q1',
    'Total_Trans_Amt',
    'Total_Trans_Ct',
    'Total_Ct_Chng_Q4_Q1',
    'Avg_Utilization_Ratio'
]


In [27]:
def perform_eda(df):
    '''
    perform eda on df and save figures to images folder
    input:
            df: pandas dataframe

    output:
            None
    '''
    df['Churn'] = df['Attrition_Flag'].apply(lambda val: 0 if val == 'Existing Customer' else 1)
    
    col_names = [
        'Churn',
        'Custom_Age',
        'Marital_Status',
        'Total_Trans_Ct',
        'heatmap'
    ]
    
    for col_name in col_names:
        plt.figure(figsize=(20,10))
        if col_name == 'Churn':
            df['Churn'].hist()
        elif col_name == 'Customer_Age':
            df['Customer_Age'].hist()
        elif col_name == 'Marital_Status':
            df['Marital_Status'].value_counts('normalize').plot(kind='bar')
        elif col_name == 'Total_Trans_Ct':
            sns.histplot(df['Total_Trans_Ct'], stat='density', kde=True)
        elif col_name == 'heatmap':
            sns.heatmap(df.corr(), annot=False, cmap='Dark2_r', linewidths=2)
        plt.title(f"{col_name}_distribution")
        plt.savefig(f"images/eda/{col_name}_distribution.png")
        plt.close()


In [28]:
def encoder_helper(df, category_lst):
    '''
    helper function to turn each categorical column into a new column with
    propotion of churn for each category - associated with cell 15 from the notebook

    input:
            df: pandas dataframe
            category_lst: list of columns that contain categorical features

    output:
            df: pandas dataframe with new columns for
    '''
    df['Churn'] = df['Attrition_Flag'].apply(lambda val: 0 if val == 'Existing Customer' else 1)
    X = pd.DataFrame()
    
    keep_cols = [
        'Churn',
        'Customer_Age',
        'Dependent_count',
        'Months_on_book',
        'Total_Relationship_Count',
        'Months_Inactive_12_mon',
        'Contacts_Count_12_mon',
        'Credit_Limit',
        'Total_Revolving_Bal',
        'Avg_Open_To_Buy',
        'Total_Amt_Chng_Q4_Q1',
        'Total_Trans_Amt',
        'Total_Trans_Ct',
        'Total_Ct_Chng_Q4_Q1',
        'Avg_Utilization_Ratio',
        'Gender_Churn',
        'Education_Level_Churn',
        'Marital_Status_Churn',
        'Income_Category_Churn',
        'Card_Category_Churn']
        
    for col_name in category_lst:
        col_lst = []
        col_groups = df.groupby(col_name).mean()['Churn']
        for val in df[col_name]:
            col_lst.append(col_groups.loc[val])
        df[f'{col_name}_Churn'] = col_lst
        
    X[keep_cols] = df[keep_cols]
    return X


In [29]:
def perform_feature_engineering(df):
    '''
    input:
              df: pandas dataframe

    output:
              X_train: X training data
              X_test: X testing data
              y_train: y training data
              y_test: y testing data
    '''
    y = df['Churn']
    X = df.drop('Churn', axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    return (X_train, X_test, y_train, y_test)


In [30]:
def classification_report_image(y_train,
                                y_test,
                                y_train_preds_lr,
                                y_train_preds_rf,
                                y_test_preds_lr,
                                y_test_preds_rf):
    '''
    produces classification report for training and testing results and stores report as image
    in images folder
    input:
            y_train: training response values
            y_test:  test response values
            y_train_preds_lr: training predictions from logistic regression
            y_train_preds_rf: training predictions from random forest
            y_test_preds_lr: test predictions from logistic regression
            y_test_preds_rf: test predictions from random forest

    output:
             None
    '''
    # save classification reprt of random forest
    plt.figure()
    plt.rc('figure', figsize=(5, 5))
    plt.text(0.01, 1.25, str('Random Forest Train'), {'fontsize':10}, fontproperties='monospace')
    plt.text(0.01, 0.05, str(classification_report(y_train, y_train_preds_rf)), {'fontsize':10}, fontproperties='monospace')
    plt.text(0.01, 0.6, str('Random Forest Test'), {'fontsize':10}, fontproperties='monospace')
    plt.text(0.01, 0.7, str(classification_report(y_test, y_test_preds_rf)), {'fontsize':10}, fontproperties='monospace')
    plt.axis('off')
    plt.savefig('./images/results/rf.png')
    plt.close()
    
    # save classification report of logistic regression
    plt.figure()
    plt.rc('figure', figsize=(5, 5))
    plt.text(0.01, 1.25, str('Logistic Regrssion Train'), {'fontsize':10}, fontproperties='monospace')
    plt.text(0.01, 0.05, str(classification_report(y_train, y_train_preds_lr)), {'fontsize':10}, fontproperties='monospace')
    plt.text(0.01, 0.6, str('Logistic Regrssion Test'), {'fontsize':10}, fontproperties='monospace')
    plt.text(0.01, 0.7, str(classification_report(y_test, y_test_preds_lr)), {'fontsize':10}, fontproperties='monospace')
    plt.axis('off')
    plt.savefig('./images/results/lr.png')
    plt.close()



In [31]:
def feature_importance_plot(model, X_data, output_pth):
    '''
    creates and stores the feature importances in pth
    input:
            model: model object containing feature_importances_
            X_data: pandas dataframe of X values
            output_pth: path to store the figure

    output:
             None
    '''
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    names = [X_data.columns[i] for i in indices]
    
    plt.figure(figsize(20,5))
    lt.title('Feature Importance')
    plt.ylabel('Importance')
    plt.bar(range(X_data.shape[1]), importances[indices])
    plt.xticks(range(X_data.shape[1], names, rotation=90))
    plt.savefig(output_pth)
    plt.close()
    


In [32]:
def train_models(X_train, X_test, y_train, y_test):
    '''
    train, store model results: images + scores, and store models
    input:
              X_train: X training data
              X_test: X testing data
              y_train: y training data
              y_test: y testing data
    output:
              None
    '''
    rfc = RandomForestClassifier(random_state=42)
    lrc = LogisticRegression(solver='lbfgs', max_iter=3000)
    
    param_grid = { 
        'n_estimators': [200, 500],
        'max_features': ['auto', 'sqrt'],
        'max_depth': [4,5,100],
        'criterion':['gini', 'entropy']
    }
    
    cv_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
    cv_rfc.fit(X_train, y_train)
    
    lrc.fit(X_train, y_train)
    
    y_train_preds_rf = cv_rfc.best_estimator_.predict(X_train)
    y_test_preds_rf = cv_rfc.best_estimator_.predict(X_test)
    
    y_train_preds_lr = lrc.predict(X_train)
    y_test_preds_lr = lrc.predict(X_test)
    
    # save the classification reports
    classification_report_image(y_train,
                                y_test,
                                y_train_preds_lr,
                                y_train_preds_rf,
                                y_test_preds_lr,
                                y_test_preds_rf)
    
    # save roc plot of rf and lr
    lrc_plot = plot_roc_curve(lrc, X_test, y_test)
    plt.figure(figsize=(15,8))
    ax = plt.gca()
    rfc_disp = plot_roc_curve(cv_rfc.best_estimator_, X_test, y_test, ax=ax, alpha=0.8)
    lrc_plot.plot(ax=ax, alpha=0.8)
    plt.savefig('./images/results/roc_result.png')
    
    # save feature importances plot
    feature_importances_plot(
        model=cv_rfc,
        X_data=X_train,
        output_pth='./images/results/feature_importance.png')
    
    # save the model
    joblib.dump(cv_rfc.best_esimatr_, './models/rfc_model.pkl')
    joblib.dump(lrc, './models/logistic_model.pkl')

In [33]:
if __name__ == '__main__':
    data_org = import_data(r'./data/bank_data.csv')
    perform_eda(data_org)
    encoded_df = encoder_helper(data_org, cat_columns)
    features = perform_feature_engineering(encoded_df)
    train_models(*features)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1